Mike Trout homered on his birthday tonight, and it is being reported as his 5th birthday HR.  How does that compare to baseball history?  Here are the leaders through 2019.

In [1]:
import pandas as pd
import calendar

In [2]:
dailies = pd.read_parquet('../data/mine/daily.parquet')
people = pd.read_parquet('../data/bd/people.parquet')[['retro_id', 'name_first', 'name_last', 'birth_month', 'birth_day']]

In [3]:
# Find games where batters homered, and add the month/day
hr_games=dailies[dailies['b_hr']>0][['game_dt', 'player_id', 'b_hr']]
hr_games['mo'] = hr_games['game_dt'].apply(lambda dt: dt.month)
hr_games['day'] = hr_games['game_dt'].apply(lambda dt: dt.day)
hr_games

,game_dt,player_id,b_hr,mo,day
10,1933-07-06,ruthb101,1.0,7,6
17,1933-07-06,frisf101,1.0,7,6
52,1935-07-08,foxxj101,1.0,7,8
68,1937-07-07,gehrl101,1.0,7,7
96,1939-07-11,dimaj101,1.0,7,11
...,...,...,...,...,...
5107018,1871-07-08,burrh101,1.0,7,8
5107021,1871-07-08,hallg101,2.0,7,8
5107023,1871-07-08,mille105,1.0,7,8
5107033,1871-07-08,selmf101,1.0,7,8


In [4]:
# Find HRs on a players birthday, by merging the daily total with people (where we have the birthday)
bd_hr = pd.merge(left=hr_games, right=people, left_on=['player_id', 'mo', 'day'], right_on=['retro_id', 'birth_month', 'birth_day'])

# Now find leaders
bd_hr_leaders = bd_hr.groupby('player_id').agg({'b_hr': sum}).sort_values(by='b_hr', ascending=False)
bd_hr_leaders_name = pd.merge(left=bd_hr_leaders, right=people, left_on='player_id', right_on='retro_id')
bd_hr_leaders_name

,b_hr,retro_id,name_first,name_last,birth_month,birth_day
0,6.0,rodra001,Alex,Rodriguez,7.0,27.0
1,6.0,reynm001,Mark,Reynolds,8.0,3.0
2,5.0,simma101,Al,Simmons,5.0,22.0
3,5.0,heltt001,Todd,Helton,8.0,20.0
4,5.0,lee-d002,Derrek,Lee,9.0,6.0
...,...,...,...,...,...,...
522,1.0,hayev001,Von,Hayes,8.0,31.0
523,1.0,harsj101,Jack,Harshman,7.0,12.0
524,1.0,harrw001,Willie,Harris,6.0,22.0
525,1.0,harrj101,John,Harris,9.0,13.0


In [5]:
# Write out the players with 4 or more birthday homers

def output_row(row):
    player_name = f'{row["name_first"]} {row["name_last"]}'
    bday = f'{calendar.month_abbr[int(row["birth_month"])]} {int(row["birth_day"]):>2}'
    hr = int(row['b_hr'])
    print(f'{player_name:<17} {hr:>3}   ({bday:<6})')
    
bd_hr_leaders_name[bd_hr_leaders_name['b_hr']>=4].apply(output_row, axis=1)
None

Alex Rodriguez      6   (Jul 27)
Mark Reynolds       6   (Aug  3)
Al Simmons          5   (May 22)
Todd Helton         5   (Aug 20)
Derrek Lee          5   (Sep  6)
Chipper Jones       5   (Apr 24)
Jason Thompson      4   (Jul  6)
Lou Gehrig          4   (Jun 19)
Aramis Ramirez      4   (Jun 25)
Nomar Garciaparra   4   (Jul 23)
Duke Snider         4   (Sep 19)
Tony Perez          4   (May 14)
Andruw Jones        4   (Apr 23)
Lance Parrish       4   (Jun 15)
Kirk Gibson         4   (May 28)
Jose Canseco        4   (Jul  2)
Gus Zernial         4   (Jun 27)
Kendrys Morales     4   (Jun 20)
Mike Trout          4   (Aug  7)
Joe Morgan          4   (Sep 19)
Justin Morneau      4   (May 15)
Garret Anderson     4   (Jun 30)
